In [1]:
import numpy as np

In [2]:
# toHex, fromHex
#    // return a 4-digit hex string corresponding to 16-bit integer n
#    public static String toHex(int n) {
#        return String.format("%04X", n & 0xFFFF);
#    }
#
#    // return a 16-bit integer corresponding to the 4-digit hex string s
#    public static int fromHex(String s) {
#        return Integer.parseInt(s, 16)  & 0xFFFF;
#    }

hex(257), int(0x101), int(0xA)

('0x101', 257, 10)

In [3]:
#    // write to an array of hex integers, 8 per line to standard output
#    public static void show(int[] a) {
#        for (int i = 0; i < a.length; i++) {
#            System.err.print(toHex(a[i]) + " ");
#            if (i % 8 == 7) System.err.println();
#        }
#    }
    

def show(a):
    return '\n'.join([ ' '.join( ["0x{:04X}".format(t) for t in a[8*k: 8*(k+1)]]) for k in range(len(a)/8 + 1)])

a = [ int(2**16*np.random.random()) for k in range(23)]
type(a[0])
print show(a)

0xBE47 0xA84B 0xDFAF 0x1B29 0xB859 0xA81C 0xB578 0x5582
0xBCDB 0x2D4F 0x4643 0x5E61 0x7067 0x8A8F 0x83C3 0xE05B
0x4154 0x3F6B 0x9254 0x38C7 0x8636 0xB4D3 0x8DC2


In [4]:
#    // write core dump of TOY to standard output
#    public static void dump(int pc, int[] R, int[] mem) {
#
#        System.err.println("PC:");
#        System.err.printf("%02X\n", pc);
#        System.err.println();
#        System.err.println("Registers:");
#        show(R);
#        System.err.println();
#        System.err.println("Main memory:");
#        show(mem);
#        System.err.println();
#    }
 
def dump( pc, R, mem):
    print "PC: %s"%(pc)
    print
    print "Registers:\n\n%s"  % (show(R  ))
    print
    print "Main memory:\n\n%s"% (show(mem))
    
def noise(N):
    return [ int(2**16*np.random.random()) for k in range(N)]

R = noise(23)
mem = noise(256)
pc = 5

dump(pc, R, mem)

PC: 5

Registers:

0x2151 0xFD09 0x7451 0x680D 0x0C11 0x93F0 0x5EC9 0x2612
0x1725 0x2773 0x0284 0x3DF5 0x1C70 0x0849 0xF3B0 0xDD2B
0x9B4E 0xE101 0x24D6 0x8E6B 0xD0CA 0xB535 0x9EEB

Main memory:

0x69A8 0xE3FC 0x64D2 0xE41F 0x1E9B 0x18B1 0x011D 0xD75C
0x9BDF 0x3F21 0xE549 0x5497 0xE9C1 0xFA04 0x36D5 0xFB87
0x2F69 0x4820 0x0627 0x5AFC 0x5CE5 0xCE5E 0xC0B1 0x124D
0xD604 0xE1DF 0x7143 0xE530 0x2AB1 0x03AA 0xA0F3 0x9DA8
0x6BD3 0xF4D1 0xF709 0xAFCF 0xD8D3 0xF2FC 0xD569 0x9633
0x96B3 0x7307 0x3485 0xA9C9 0x9E79 0x8EF1 0xD094 0x968C
0xACDD 0xCF18 0xC968 0x919A 0x93A3 0x9157 0x8BE7 0xF18D
0x4BA5 0x1995 0xC9FC 0x0AE0 0xBE3E 0x6666 0x989F 0x79A6
0x1157 0xBC45 0x226D 0x7BFD 0xF6AA 0x05D9 0xDBCF 0xAB96
0xFCF4 0x5995 0x1C91 0x1223 0x9682 0x25E4 0x29B2 0xC8B1
0x6555 0x898F 0x7DE1 0x6D7A 0xC726 0xBADD 0x1DA9 0x1C62
0x8888 0x4DEC 0x0058 0xCE28 0x0FE4 0xD12F 0xB258 0xE3A2
0x8AD2 0xE21B 0x5DA8 0xC17B 0xACE2 0x6B3A 0xCA30 0xC68F
0x91E8 0xC91D 0x7F1D 0x24BF 0x3839 0x68CA 0xE536 0x408A
0x8E8C 0xBD6F 0x575B 

In [12]:
# ftp://ftp.cs.princeton.edu/pub/cs126/hamming/multiply.toy

program = """
10: 8AFF   read R[A]                     a = StdIn.readInt();
11: 8BFF   read R[B]                     b = StdIn.readInt();

12: 7C00   R[C] <- 0000                  c = 0;
13: 7101   R[1] <- 0001                  the constant 1

14: CA18   if (R[A] == 0) goto 18        while (a != 0) {
15: 1CCB   R[C] <- R[C] + R[B]              c += b;
16: 2AA1   R[A] <- R[A] - R[1]              a -= 1;
17: C014   goto 14                       }

18: 9CFF   write R[C]                    StdOut.println(c);      
19: 0000   halt                          """

program = [ x[:8].split(":") for x in program.split("\n") if len(x) > 0]

program = [ (int(x[0], 16), int(x[1][1:], 16)) for x in program]

program

pc  = 0x10
R   = [0 for x in range(16)]
mem = [0 for x in range(256)]

for x in program:
    mem[x[0]] = x[1]
    
#print "Core Dump of TOY Before Executing"
#print "---------------------------------"
#dump(pc, R, mem)

# Fetch and parse

while True:
    # fetch next instruction
    inst = mem[pc]; pc += 1
    # get opcode (bits 12-15)
    op   = (inst >> 12) &  15    
    # get dest   (bits  8-11)
    d    = (inst >>  8) &  15    
    # get s      (bits  4- 7)
    s    = (inst >>  4) &  15    
    # get t      (bits  0- 3)
    t    = (inst >>  0) &  15 
    # get addr   (bits  0- 7)
    addr = (inst >>  0) & 255    

    print "%s\t%s\t%s\t%s\t%s\t"  % ((" " + str(op))[-2:], 
                                   (" " + str(d ))[-2:], 
                                   (" " + str(s ))[-2:], 
                                   (" " + str(t ))[-2:], 
                                   (".." + str(addr))[-3:]),

    if op == 0: break
        
    # stdin
    if ((addr == 255 and op == 8) or (R[t] == 255 and op == 10)):
        mem[255] = int(input(), 16);
    
   
    
    # execute
    # https://wiki.python.org/moin/BitwiseOperators
    if op == 1:
        R[d] = R[s] + R[t]
    if op == 2:
        R[d] = R[s] - R[t]
    if op == 3:
        R[d] = R[s] & R[t]    
    if op == 4:
        R[d] = R[s] ^ R[t]
    if op == 5:
        R[d] = R[s] << R[t]    
    if op == 6:
        R[d] = R[s] >> R[t]
    if op == 7:
        R[d] = addr    
    if op == 8:
        R[d] = mem[addr]
    if op == 9:
        mem[addr] = R[d]   
    if op ==10:
        R[d] = mem[R[t] & 255]       
    if op ==11:
        mem[R[t] & 255] = R[d]
    if op ==12:
        if R[d] == 0:
            pc = addr
    if op ==13:
        if R[d]  > 0:
            pc = addr    
    if op ==14:
        pc = R[d]
    if op ==15:
        R[d] = pc
        pc   = addr
    #print R
        
    #  stdout
    if ((addr == 255 and op == 9) or (R[t] == 255 and op == 11)):
        print "\t" + "0x{:04X}".format(mem[255])
    else:
        print
        

    R[0] = 0
    R[d] = R[d] & 0xFFFF
    pc   = pc   & 0xFF
    
    print pc,
    
print 
print
print "Core Dump of TOY After Executing"
print "--------------------------------"
dump(pc, R, mem)

 8	10	15	15	255	"7"

17  8	11	15	15	255	"9"

18  7	12	 0	 0	..0	
19  7	 1	 0	 1	..1	
20 12	10	 1	 8	.24	
21  1	12	12	11	203	
22  2	10	10	 1	161	
23 12	 0	 1	 4	.20	
20 12	10	 1	 8	.24	
21  1	12	12	11	203	
22  2	10	10	 1	161	
23 12	 0	 1	 4	.20	
20 12	10	 1	 8	.24	
21  1	12	12	11	203	
22  2	10	10	 1	161	
23 12	 0	 1	 4	.20	
20 12	10	 1	 8	.24	
21  1	12	12	11	203	
22  2	10	10	 1	161	
23 12	 0	 1	 4	.20	
20 12	10	 1	 8	.24	
21  1	12	12	11	203	
22  2	10	10	 1	161	
23 12	 0	 1	 4	.20	
20 12	10	 1	 8	.24	
21  1	12	12	11	203	
22  2	10	10	 1	161	
23 12	 0	 1	 4	.20	
20 12	10	 1	 8	.24	
21  1	12	12	11	203	
22  2	10	10	 1	161	
23 12	 0	 1	 4	.20	
20 12	10	 1	 8	.24	
24  9	12	15	15	255		0x003F
25  0	 0	 0	 0	..0	

Core Dump of TOY After Executing
--------------------------------
PC: 26

Registers:

0x0000 0x0001 0x0000 0x0000 0x0000 0x0000 0x0000 0x0000
0x0000 0x0000 0x0000 0x0009 0x003F 0x0000 0x0000 0x0000


Main memory:

0x0000 0x0000 0x0000 0x0000 0x0000 0x0000 0x0000 0x0000
0x0000 0x0000 0x0

In [72]:
int('8AFF', 16), int('0x8AFF', 16)

(35583, 35583)

In [9]:
R

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0]